簡易計算with時間構造化NW

In [2]:
import pandas as pd 
import os
import numpy as np 
import networkx as nx
from scipy.stats import norm
from scipy.optimize import minimize

In [3]:
# 目的地を16だけにして，リンクデータの構成は全く同じにしてやってみる
df_link = pd.read_csv('/Users/takahiromatsunaga/res2023/easy_test/testnw_link2.csv')
df_node = pd.read_csv('/Users/takahiromatsunaga/res2023/easy_test/testnw_node.csv')
df_data = pd.read_csv('/Users/takahiromatsunaga/res2023/easy_test/pydata2.csv')
ODlist = pd.read_csv('/Users/takahiromatsunaga/res2023/easy_test/ODlist.csv')

In [4]:
####### リンク接続行列（無向） ####### 
def I_nod(link_data): 
    n = len(link_data)
    I = np.eye(n) # 滞在リンク（時間構造化する場合は吸収リンクにも滞在可能リンクつける）
    for i in range(n):
        O = link_data.loc[i, 'o'] # 当該linkのOnode
        D = link_data.loc[i, 'd'] # 当該linkのDnode
        for j in range(n): # リンクiのOorDをOないしDとしているリンクjとは接続
            if ((link_data.loc[j, 'o'] == O) or (link_data.loc[j, 'o'] == D)) or (link_data.loc[j, 'd'] == O) or (link_data.loc[j, 'd'] == D): 
                I[i, j] = 1
    return(I)

I = I_nod(df_link)

#### 少なくとも時空間プリズムで吸収から別のリンクに行く，みたいなのはおかしいので
I[15, :15] = 0

# 時間構造化する場合は接続情報はそのまま？？？？？吸収リンクからはでも吸収リンク以外に伸ばしていいのか？
# 時間構造化する場合はt
#ddata = [16, 17] 
ddata = [16] 

####### パラメタ初期設定 #######
xdim = 2
x0 = np.zeros(xdim)
x0[0] = -1
x0[-1] = 0.8

In [25]:
T = 6 # 6は吸収に入っているときの時刻
L = len(df_link)

Ilist = []
for i in range(L):
    I[i, i] = 1 # Iは普通の空間接続行列．これに滞在リンクを追加している
    
# 起点Oからの到達可能性指示変数ioの定義
Itlist = np.zeros((T+1, L, L))
II = np.copy(I)
Itlist[0, :, :] = np.eye(L) # ItlistにはT+1成分あるが，その第一成分(t=0)は単位行列ということ．t=1以降はfor内で計算

for ts in range(1, T+1):
    Itlist[ts, :, :] = II # 時刻1では普通の接続行列
    II = np.dot(II, I) # 接続条件の積．つまり2回先に接続していたら1，そうでなければ0
    II = np.where(II > 0, 1, 0) 
        # この時点ではItlistには時刻0にスタートして時刻tsに利用可能なリンクが1になっているような行列が各時刻毎に入っている

# 終点Dからの到達可能性指示変数idの定義．時刻tにkまで来てる時に，スタートして利用可能なリンクaが1になってる？
Ittlist = np.zeros((T+1, L, L))
for ts in range(T+1):
    Ittlist[ts, :, :] = np.transpose(Itlist[T - ts, :, :]) 
#### Ittlistは時刻が増えるたびに0が増えていく．kをd（t=T）とするとき時刻tに滞在可能なリンクaが1になっている???→多分そうだ
#### Itlistは時刻が増えるたびに1が増えていく．kをo（t=0）にしたときに時刻t内で到達可能なリンクaが1になっている
print(Itlist[1]) # 
print(Ittlist[0]) # Ittlist[0]はIttlist[0, :-1, -1]=0, それ以外は1→
# Ittlist[T]は対角成分（それはそう）

[[1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1

In [32]:
# 時空間プリズムは個人の一品生産．
nod = len(ODlist)

T=6
L = len(df_link)



###### 時空間プリズム準備 ######
Ilist = []
for i in range(L):
    I[i, i] = 1 # Iは普通の空間接続行列．これに滞在リンクを追加している
    
# 起点Oからの到達可能性指示変数ioの定義 # わかったわ．
Itlist = np.zeros((T+1, L, L))
II = np.copy(I)
Itlist[0, :, :] = np.eye(L) # ItlistにはT+1成分あるが，その第一成分(t=0)は単位行列ということ．t=1以降はfor内で計算

for ts in range(1, T+1):
    Itlist[ts, :, :] = II # 時刻1では普通の接続行列
    II = np.dot(II, I) # 接続条件の積．つまり2回先に接続していたら1，そうでなければ0
    II = np.where(II > 0, 1, 0) 

# 終点Dからの到達可能性指示変数idの定義
Ittlist = np.zeros((T+1, L, L))
for ts in range(1, T+2):
    Ittlist[ts-1, :, :] = np.transpose(Itlist[(T+1) - ts, :, :]) 


for od in range(nod): # odのループ内におくならいいかな→ODのループが正しい
    Id = np.zeros((T, L, L))
    # print(ODlist.loc[od, 'o'])
    o = ODlist.loc[od, 'o'] - 1 # 多分
    d = ODlist.loc[od, 'd'] - 1
    #o = 12-1 # 理論的には1リンクだけ寄り道か滞在が可能
    #d = 14-1 # 吸収ではなく最後の実態リンクを目的地としている

    #この前にod与えるのでt=0入ったっmm実行しておき
    for ts in range(T):

        # ts=0で全部の接続リンクが採用されてるの納得いかないが．
        if ts == 0:
            Id[ts, o, :] = I[o, :] # 最初はoの接続行
            # print(f'個人{od+1}の最初は{o+1}') # ここはちゃんと読めている
            continue
        
        # まずはo条件Itlist＆d条件Ittlist下で次の時刻に利用可能or不可が揃うようなリンクが候補alistになる
        alist = np.where(Itlist[ts + 1, o, :] == Ittlist[ts + 1, d, :])[0]  
        # alistはts+1において0か1が等しいaのリスト．
        # 次の時間の遷移先候補を絞っているのでts+1で指定するのは正しい→その結果をId[t]に入れるのは正しい．
        print(f'userid{od+1}のoは{o+1}でdは{d+1}，この人の時刻{ts+1}でのalistは{alist}')


        # 
        for a in alist:
            if Itlist[ts + 1, o, a] == Ittlist[ts + 1, d, a]: # これは毎回成り立つのではと思うが．1か0かは指定してないのでダメじゃね？？→0だったらどうするの
                # 次に利用可能なリンクがa．これに接続するようなkが候補となる．
                print(f'これ0じゃ不味くね{Itlist[ts + 1, o, a]}')
                #### 後ろの方が，0の場合は落とされてるので処理的にはOKと言うことになる！！！
                klist = np.where(I[:, a] == 1)[0] # aに接続しているk（上ではkとか考えてなかったので）

                for k in klist:
                    if len(np.where(Id[ts - 1, :, k] == 1)[0]) != 0:
                        Id[ts, k, a] = 1 # 
                        print(f'個人{od+1}の時刻{ts}で{k+1}から{a+1}は利用可能')
                    else:
                        print(f'{k+1}が落とされた')
            #else:
                #print(f'そんなわけなくね') # 実際，そんなことはなかったです，流石にねw

    Ilist.append(Id) # 個人毎にIlistに入れていく

# Ilist[i]には個人iの時空間プリズムが入っている
Id = Ilist[10]
#print(Id[4, :, :]) # Id[t, :, :]には，時刻tに各行にいた時，次に利用可能なリンクが入ってる（1になっている）．この時点で相当リンク絞られている印象
# 時刻3にはもう8にいくしかない．2で8にいたら6, 7も利用可能
# odを与えてIdを個人毎に作成するのがまず重要．od与えるのはqallの最初と最後見て
# 続いて各時刻でId[t, :, :]を引用
# cand_listを今は接続している全リンクをIからIから取って来ていたが，これを時間制約で絞り込めるはずなのでId[t, at_1, :](tは時間のカウント)で大幅に絞り込む

userid1のoは1でdは16，この人の時刻2でのalistは[ 0  1  2  3  4  5  6  9 10 11 12 13]
これ0じゃ不味くね1.0
個人1の時刻1で1から1は利用可能
個人1の時刻1で2から1は利用可能
個人1の時刻1で5から1は利用可能
これ0じゃ不味くね1.0
個人1の時刻1で1から2は利用可能
個人1の時刻1で2から2は利用可能
3が落とされた
個人1の時刻1で5から2は利用可能
6が落とされた
これ0じゃ不味くね1.0
個人1の時刻1で2から3は利用可能
3が落とされた
4が落とされた
6が落とされた
これ0じゃ不味くね0.0
3が落とされた
4が落とされた
これ0じゃ不味くね1.0
個人1の時刻1で1から5は利用可能
個人1の時刻1で2から5は利用可能
個人1の時刻1で5から5は利用可能
7が落とされた
14が落とされた
これ0じゃ不味くね1.0
個人1の時刻1で2から6は利用可能
3が落とされた
6が落とされた
7が落とされた
8が落とされた
これ0じゃ不味くね1.0
個人1の時刻1で5から7は利用可能
6が落とされた
7が落とされた
8が落とされた
14が落とされた
これ0じゃ不味くね0.0
10が落とされた
11が落とされた
これ0じゃ不味くね0.0
9が落とされた
10が落とされた
11が落とされた
12が落とされた
これ0じゃ不味くね0.0
9が落とされた
11が落とされた
12が落とされた
13が落とされた
これ0じゃ不味くね0.0
12が落とされた
13が落とされた
これ0じゃ不味くね1.0
個人1の時刻1で5から14は利用可能
7が落とされた
14が落とされた
userid1のoは1でdは16，この人の時刻3でのalistは[ 0  1  4  5  6  7  8  9 10 11 12 13 14 15]
これ0じゃ不味くね1.0
個人1の時刻2で1から1は利用可能
個人1の時刻2で2から1は利用可能
個人1の時刻2で5から1は利用可能
これ0じゃ不味くね1.0
個人1の時刻2で1から2は利用可能
個人1の時刻2で2から2は利用可能
個人1の時刻2で3から2は利用可能
個人1の時刻2で5から2は利用可能
個人1の時刻2で6から2は利用可能
これ0じゃ不味くね1.0
個人1の時刻2で1から5は利用可能
個

In [15]:
def Mset(x): 
    cost = np.exp(df_link['length'] * x[0]) #  + df_link['staire'] * x[1]) # + df_link_integrated['staire_with_esc'] * x[3])
    cost = pd.concat([cost]*L, axis=1)
    cost = cost.T
    cost_numpy = cost.values # DataFrameをNumPy配列に変換
    return cost_numpy

# 以降石井さんのコード参考に書き下す
def newV(x):
    beta = x[-1]
    z = np.ones((T+1, L, nod))
    for od in range(nod): # 個人毎
        M = np.zeros((T, L, L))
        Id = Ilist[od] # Id[t]のk行は，時刻tにリンクkにいた時に利用可能な遷移先リンクが1になっている
        for ts in range(T):
            Mts = Id[ts, :, :] * Mset(x)
            M[ts, :, :] = Mts
        
        for tt in range(T-1, -1, -1):
            #print(f'個人{od+1}の時刻{tt+1}でのM{M[tt, :, :]},zは{z[tt+1, :, od]}')
            zi = np.dot(M[tt, :, :], (z[tt+1, :, od] ** beta))
            z[tt, :, od] = np.where(zi == 0, 1, zi)
    
    return z


# newV不要
def loglikelihood(x):
    beta = x[-1]
    LL = 0
    #z = newV(x)
    #V = np.log(z)
    
    for od in range(nod):
        Id = Ilist[od] # 個人odの時空間プリズム
        M = np.zeros((T, L, L))

        for ts in range(T):
            Mts = Id[ts, :, :] * Mset(x) # 時刻tsにおいて，次につながっているリンクたちに対して初めて遷移効用が与えられる．なのでIdは時刻tsにおける接続条件（kがts, aがts+1のリンクで，k→aの遷移が可能か否か
            M[ts, :, :] = Mts

        ## 個人の各時点毎の遷移効用行列（時間のプリズムによる）が書けた

        z = np.ones((T+1, L))
        for t in range(T, 0, -1):
            zii = M[t-1, :, :] * (z[t, :] ** beta)
            zi = zii.sum(axis = 1)
            z[t-1, :] = (zi==0)*1 + (zi!=0)*zi
        
        print(z)
        
        knode = list(df_data.loc[od*T:(od+1)*T-1, 'k'])
        anode = list(df_data.loc[od*T:(od+1)*T-1, 'a'])

        # print(f'個人{od+1}でknodeは{knode},anodeは{anode}') # ここはちゃんと出てる
        # 16にいる間は将来効用1のままで割り引かれないのがいい
        for t in range(T):
            k = int(knode[t]) - 1 
            a = int(anode[t]) - 1
            
            p = np.exp(np.log(M[t, k, :]) + beta * np.log(z[t+1, :]) - np.log(z[t, k]))
            
            #print(f'個人{od+1}の時刻{t+1}の遷移は{k+1}から{a+1}で，その即時効用は{M[t, k, a]}，{a+1}の期待効用は{z[t+1, a]}')
            #print(p)
            
            LL += np.log(M[t, k, a]) + beta * np.log(z[t+1, a]) - np.log(z[t, k])

    return -LL

#x0 = np.array((-1, 1))
#print(loglikelihood(x0))

#Mset(x0)

In [16]:
x = np.array((-2, 0.6))
print(loglikelihood(x))

[[0.16445895 1.         1.         1.         1.         1.
  1.         1.         1.         1.         1.         1.
  1.         1.         1.         1.        ]
 [0.13630011 0.1928311  1.         1.         0.35834525 1.
  1.         1.         1.         1.         1.         1.
  1.         1.         1.         1.        ]
 [0.09535958 0.11520068 0.03968219 1.         0.30408883 0.11520068
  0.30408883 1.         1.         1.         1.         1.
  1.         1.26440664 1.         1.        ]
 [0.0407622  0.0407622  1.         1.         0.22756901 0.0407622
  0.22756901 0.0407622  1.         1.         1.         1.
  1.         1.22756901 1.         1.        ]
 [1.         1.         1.         1.         0.13533528 1.
  0.13533528 1.         1.         1.         1.         1.
  1.         1.13533528 1.         1.        ]
 [1.         1.         1.         1.         1.         1.
  1.         1.         1.         1.         1.         1.
  1.         1.         1.    

/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_66926/1745122732.py:61: RuntimeWarning: divide by zero encountered in log
  p = np.exp(np.log(M[t, k, :]) + beta * np.log(z[t+1, :]) - np.log(z[t, k]))


[[1.         1.         1.         1.         1.         1.
  1.         1.         1.         1.         1.         1.
  0.14527517 1.         1.         1.        ]
 [1.         1.         1.         1.         1.         1.
  1.         1.         1.         1.         1.         0.01288102
  1.         1.         1.         1.        ]
 [1.         1.         1.         1.         1.         1.
  1.         1.         0.01984109 1.         1.         1.
  1.         1.         1.         1.        ]
 [1.         1.         1.         1.         1.         1.
  1.         0.0407622  1.         1.         1.         1.
  1.         1.         1.         1.        ]
 [1.         1.         1.         1.         1.         1.
  0.13533528 1.         1.         1.         1.         1.
  1.         1.         1.         1.        ]
 [1.         1.         1.         1.         1.         1.
  1.         1.         1.         1.         1.         1.
  1.         1.         1.         1.

In [144]:
###### 推定部分 ###### 
dL = 100
n = 0

x_init = np.zeros(2)
bounds = [(-25, 0), (0, 1)] # , (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (-10, 10), (0, 1)]
# beta = 1
# 注意．LLを返す関数frに対して数値微分を実行
def fr(x): 
    return -loglikelihood(x)

def hessian(x: np.array) -> np.array:
    h = 10 ** -4 # 数値微分用の微小量
    n = len(x)
    res = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            e_i, e_j = np.zeros(n), np.zeros(n)
            e_i[i] = 1
            e_j[j] = 1
            
            res[i][j] = (fr(x + h * e_i + h * e_j)
            - fr(x + h * e_i - h * e_j)
            - fr(x - h * e_i + h * e_j)
            + fr(x - h * e_i - h * e_j)) / (4 * h * h)
    return res

def tval(x: np.array) -> np.array:
    print(f'hesse行列の逆行列{np.linalg.inv(hessian(x))}')            # inv逆行列を計算！
    print(f'各パラメタの分散{-np.diag(np.linalg.inv(hessian(x)))}')        # 対角成分取り出して-1倍
    return x / np.sqrt(-np.diag(np.linalg.inv(hessian(x))))

res = minimize(loglikelihood, x, method='L-BFGS-B', bounds = bounds) #, options={"maxiter":10,"return_all":True}) 
        
# 推定値でパラメタxを更新
x0 = res.x

### hesse行列自作バージョン

hhh = hessian(x0)
print(hhh)
print(f'繰り返し回数{n}')
#tval = x0 / np.sqrt(np.diag(hhh))
tval = tval(x0)

### BFGS
"""
hhh = res.hess_inv #.todense()
    # print(n)
tval = x0 / np.sqrt(np.diag(hhh))
"""

L0 = -1 * loglikelihood(x_init) # 初期尤度
LL = -1 * res.fun

#end_time = time.time()
#proc_time = end_time - start_time

    ###### 最終結果の出力 ######
print("計算時間")
#print(proc_time)
print("Inputdata")
#print(search_folder)
print("結果の表示")
print(res)
    #print("説明変数の表示")
    #print(linkv)
    #print("NFXP")
print("初期尤度 = ", L0)
print("最終尤度 = ", LL)
print("ρ値 = ", (L0 - LL) / L0)
print("修正済ρ値 = ", (L0 - (LL - len(x0))) / L0)
print("パラメータ初期値 = ", x_init)
print("パラメータ推定値 = ", x0)
print("時間割引率 = ", x0[-1])
print("t値 = ", tval)

# beta=0.6にすると最適値は-8.580になった（下限-25で設定）
# beta=0.8にすると-6.12だ
# beta=0.9にすると-5.006だ
# beta=1にすると初期尤度nanになる-4.18で出た←beta1は信用できない．
# betaも推定させてみる．上限0.9に張り付いた

# beta上限0.99にしてみる．→やはりbeta=0.99に張り付く．
"""
初期尤度 =  -273.47889537719175
最終尤度 =  -88.76375392630297
ρ値 =  0.6754274080130502
修正済ρ値 =  0.6681142294321527
パラメータ初期値 =  [0. 0.]
パラメータ推定値 =  [-4.25710943  0.99      ]
時間割引率 =  0.99
t値 =  [-4.78393893  8.57774557]
"""
# beta上限1にすると崩壊しそうだけど．
"""
初期尤度 =  -273.47889537719175
最終尤度 =  -88.41899201904533
ρ値 =  0.6766880607108833
修正済ρ値 =  0.6693748821299856
パラメータ初期値 =  [0. 0.]
パラメータ推定値 =  [-4.18538293  1.        ]
時間割引率 =  1.0
t値 =  [-4.73587612  8.46022775]
"""
# 1には張り付いたものの尤度はnanになってない．よくわからないすぎる．

# 目的地二つ入れると．beta上限は1．t値出過ぎているけど
"""
)
  message: 'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 63
      nit: 0
     njev: 21
   status: 2
  success: False
        x: array([-5.88086299,  1.        ])
初期尤度 =  -309.7573152401913
最終尤度 =  -47.681145408866385
ρ値 =  0.8460693482835311
修正済ρ値 =  0.8396126807518889
パラメータ初期値 =  [0. 0.]
パラメータ推定値 =  [-5.88086299  1.        ]
時間割引率 =  1.0
t値 =  [-169957.20106105   28900.84066295]

"""

/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_30336/325132223.py:61: RuntimeWarning: divide by zero encountered in log
  p = np.exp(np.log(M[t, k, :]) + beta * np.log(z[t+1, :]) - np.log(z[t, k]))


[[  -2.22938912   21.28907202]
 [  21.28907202 -275.11534402]]
繰り返し回数0
hesse行列の逆行列[[-1.71824057 -0.13296149]
 [-0.13296149 -0.01392371]]
各パラメタの分散[1.71824057 0.01392371]
計算時間
Inputdata
結果の表示
      fun: 60.570217462192126
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-2.13162822e-06, -5.61328762e-05])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 42
      nit: 11
     njev: 14
   status: 0
  success: True
        x: array([-3.83662465,  0.47607809])
初期尤度 =  -100.71964815521693
最終尤度 =  -60.570217462192126
ρ値 =  0.3986256051168027
修正済ρ値 =  0.37876850636167353
パラメータ初期値 =  [0. 0.]
パラメータ推定値 =  [-3.83662465  0.47607809]
時間割引率 =  1.0
t値 =  [-2.92689625  4.03460178]


"\n)\n  message: 'ABNORMAL_TERMINATION_IN_LNSRCH'\n     nfev: 63\n      nit: 0\n     njev: 21\n   status: 2\n  success: False\n        x: array([-5.88086299,  1.        ])\n初期尤度 =  -309.7573152401913\n最終尤度 =  -47.681145408866385\nρ値 =  0.8460693482835311\n修正済ρ値 =  0.8396126807518889\nパラメータ初期値 =  [0. 0.]\nパラメータ推定値 =  [-5.88086299  1.        ]\n時間割引率 =  1.0\nt値 =  [-169957.20106105   28900.84066295]\n\n"

In [72]:
"""
def loglikelihood(x):
    beta = x[-1]
    LL = 0
    z = newV(x)
    V = np.log(z)

    for od in range(nod):
        Id = Ilist[od] # 個人odの時空間プリズム
        #M = 
        #zod = z[]

        for t in range(T):
            # 
            LL += np.log(M[k,a]) + beta*np.log(zod[t+1,a]) - np.log(zod[t,k])
    return -LL
"""

'\ndef loglikelihood(x):\n    beta = x[-1]\n    LL = 0\n    z = newV(x)\n    V = np.log(z)\n\n    for od in range(nod):\n        Id = Ilist[od] # 個人odの時空間プリズム\n        #M = \n        #zod = z[]\n\n        for t in range(T):\n            # \n            LL += np.log(M[k,a]) + beta*np.log(zod[t+1,a]) - np.log(zod[t,k])\n    return -LL\n'

In [ ]:
import numpy as np

# サンプルデータ
L = 5
Itlist = np.random.randint(0, 2, size=(5, L, L)) # timestep5, link数10
Ittlist = np.random.randint(0, 2, size=(5, L, L))

print(Itlist)
print(Ittlist)

ts = 3 # 4番目の時刻においてoが2つまりindex1, dが4つまりindexが3
o = 2
d = 4

# 条件に基づいてインデックスを抽出する
alist = np.where(Itlist[ts + 1, o, :] == Ittlist[ts + 1, d, :])[0] + 1
# R言語の1-based indexをPythonの0-based indexに変換するために +1 をしています

print(alist)


In [53]:
c = np.array([[(3, 4, 5), (2, 3, 4), (6, 7, 22)], [(8, 222, 15), (21, 43, 9), (3, 214, 2)]])
# c[1, :, 1] の取得
result = c[1, :, 1] # 第二成分の各タプルから，第二成分を取り出して並べたもの
print(result)

[222  43 214]
